In [72]:
# Importar bibliotecas necessárias
import pandas as pd
import re
from conexao_bancodedados import *

In [131]:
# Definir o caminho do arquivo CSV
csv_file = r'C:\Users\Lucas\OneDrive\Documentos\projetos_python\Jornada_financas_pessoais\data\Controle de Ações e FII - Yasmin - FII.csv'

# Pega o nome do acionista que está entre os "- -"
nome_acionista = re.search(r'- (.*) -', csv_file).group()

# Ler o arquivo CSV delimitado por vírgula
arquivo = pd.read_csv(csv_file, sep=',', names=['DATA', 'PAPEL', 'OP', 'QTD', 'PRECO', 'CUSTO_TOTAL', 'TOTAL_OP', 'RAT', 'VL_LIQUIDO', 'ESTOQUE', 'P_MEDIO', 'GAN_PER', 'IR_MES', 'MES_REF', 'VENDAS_MES'], skiprows=1)

# Transforma arquivo em um DataFrame para que possa ser aplicado os métodos
df = pd.DataFrame(arquivo)

In [ ]:
# Inclui nome do acionista
df['Acionista'] = nome_acionista

# Remove "-" que estão entre a coluna nome do acionista
df['Acionista'] = df['Acionista'].str.replace('-', '')

# Exibe Dataframe 
df.head()

In [128]:
# Truncar os dados da tabela antes de carregar
truncate_table('bronze','controle_ativo')

In [132]:
# Conexão com o banco de dados na camada "bronze":
engine = conexao_mysql('bronze')

df.to_sql(name='controle_ativo', con=engine, if_exists='append', index=False)

-1